In [1]:
import sqlite3
import json
from datetime import datetime
import time

timeframe = '2015-05'
sql_transaction = []
start_row = 0
cleanup = 1000000

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False
    
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0

    #with open('J:/chatdata/reddit_data/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
    with open('../2015-05-data/RC_{}'.format(timeframe), buffering=1000) as f:
        for row in f:
            #print(row)
            #time.sleep(555)
            row_counter += 1

            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if acceptable(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

            if row_counter > start_row:
                if row_counter % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()

Total Rows Read: 100000, Paired Rows: 5318, Time: 2020-05-23 08:48:33.015919
Total Rows Read: 200000, Paired Rows: 13009, Time: 2020-05-23 08:48:50.690186
Total Rows Read: 300000, Paired Rows: 21577, Time: 2020-05-23 08:49:08.999839
Total Rows Read: 400000, Paired Rows: 30679, Time: 2020-05-23 08:49:27.581595
Total Rows Read: 500000, Paired Rows: 39954, Time: 2020-05-23 08:49:46.482005
Total Rows Read: 600000, Paired Rows: 48738, Time: 2020-05-23 08:50:05.774349
Total Rows Read: 700000, Paired Rows: 57043, Time: 2020-05-23 08:50:24.908113
Total Rows Read: 800000, Paired Rows: 65839, Time: 2020-05-23 08:50:44.015427
Total Rows Read: 900000, Paired Rows: 75385, Time: 2020-05-23 08:51:02.938227
Total Rows Read: 1000000, Paired Rows: 85394, Time: 2020-05-23 08:51:21.642798
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 93581, Time: 2020-05-23 08:51:48.678383
Total Rows Read: 1200000, Paired Rows: 102975, Time: 2020-05-23 08:52:07.462824
Total Rows Read: 1300000, Paired Rows: 112878, Ti

Total Rows Read: 10300000, Paired Rows: 944931, Time: 2020-05-23 09:23:45.926006
Total Rows Read: 10400000, Paired Rows: 954983, Time: 2020-05-23 09:24:05.523716
Total Rows Read: 10500000, Paired Rows: 964928, Time: 2020-05-23 09:24:25.132163
Total Rows Read: 10600000, Paired Rows: 974434, Time: 2020-05-23 09:24:44.055093
Total Rows Read: 10700000, Paired Rows: 983489, Time: 2020-05-23 09:25:02.836544
Total Rows Read: 10800000, Paired Rows: 992670, Time: 2020-05-23 09:25:21.313531
Total Rows Read: 10900000, Paired Rows: 1001971, Time: 2020-05-23 09:25:39.901040
Total Rows Read: 11000000, Paired Rows: 1011384, Time: 2020-05-23 09:25:58.728768
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 1018893, Time: 2020-05-23 09:26:45.770795
Total Rows Read: 11200000, Paired Rows: 1027150, Time: 2020-05-23 09:27:05.752012
Total Rows Read: 11300000, Paired Rows: 1035722, Time: 2020-05-23 09:27:25.438662
Total Rows Read: 11400000, Paired Rows: 1045174, Time: 2020-05-23 09:27:45.132007
Total Rows

Total Rows Read: 20200000, Paired Rows: 1870358, Time: 2020-05-23 10:02:34.740792
Total Rows Read: 20300000, Paired Rows: 1880432, Time: 2020-05-23 10:02:54.342593
Total Rows Read: 20400000, Paired Rows: 1890539, Time: 2020-05-23 10:03:13.833601
Total Rows Read: 20500000, Paired Rows: 1900928, Time: 2020-05-23 10:03:33.300156
Total Rows Read: 20600000, Paired Rows: 1911231, Time: 2020-05-23 10:03:52.415524
Total Rows Read: 20700000, Paired Rows: 1921453, Time: 2020-05-23 10:04:11.387351
Total Rows Read: 20800000, Paired Rows: 1931738, Time: 2020-05-23 10:04:30.865152
Total Rows Read: 20900000, Paired Rows: 1941993, Time: 2020-05-23 10:04:50.086928
Total Rows Read: 21000000, Paired Rows: 1952272, Time: 2020-05-23 10:05:10.563484
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 1959963, Time: 2020-05-23 10:06:21.147474
Total Rows Read: 21200000, Paired Rows: 1968768, Time: 2020-05-23 10:06:40.538973
Total Rows Read: 21300000, Paired Rows: 1977188, Time: 2020-05-23 10:07:00.049077
Tota

Total Rows Read: 30100000, Paired Rows: 2794082, Time: 2020-05-23 10:45:14.652561
Total Rows Read: 30200000, Paired Rows: 2803544, Time: 2020-05-23 10:45:35.286904
Total Rows Read: 30300000, Paired Rows: 2812699, Time: 2020-05-23 10:45:55.912577
Total Rows Read: 30400000, Paired Rows: 2821852, Time: 2020-05-23 10:46:16.358278
Total Rows Read: 30500000, Paired Rows: 2830896, Time: 2020-05-23 10:46:36.272952
Total Rows Read: 30600000, Paired Rows: 2840532, Time: 2020-05-23 10:46:56.247783
Total Rows Read: 30700000, Paired Rows: 2850356, Time: 2020-05-23 10:47:16.325159
Total Rows Read: 30800000, Paired Rows: 2860395, Time: 2020-05-23 10:47:36.113546
Total Rows Read: 30900000, Paired Rows: 2870611, Time: 2020-05-23 10:47:56.085566
Total Rows Read: 31000000, Paired Rows: 2880989, Time: 2020-05-23 10:48:14.935231
Cleanin up!
Total Rows Read: 31100000, Paired Rows: 2889348, Time: 2020-05-23 10:49:50.569702
Total Rows Read: 31200000, Paired Rows: 2899449, Time: 2020-05-23 10:50:11.514561
Tota

Total Rows Read: 40000000, Paired Rows: 3719186, Time: 2020-05-23 11:30:15.393522
Cleanin up!
Total Rows Read: 40100000, Paired Rows: 3727052, Time: 2020-05-23 11:32:07.079658
Total Rows Read: 40200000, Paired Rows: 3736438, Time: 2020-05-23 11:32:27.003835
Total Rows Read: 40300000, Paired Rows: 3746010, Time: 2020-05-23 11:32:46.872297
Total Rows Read: 40400000, Paired Rows: 3755559, Time: 2020-05-23 11:33:05.987689
Total Rows Read: 40500000, Paired Rows: 3765285, Time: 2020-05-23 11:33:25.637779
Total Rows Read: 40600000, Paired Rows: 3774587, Time: 2020-05-23 11:33:45.287234
Total Rows Read: 40700000, Paired Rows: 3783345, Time: 2020-05-23 11:34:04.231674
Total Rows Read: 40800000, Paired Rows: 3792386, Time: 2020-05-23 11:34:23.065598
Total Rows Read: 40900000, Paired Rows: 3802003, Time: 2020-05-23 11:34:42.398502
Total Rows Read: 41000000, Paired Rows: 3810948, Time: 2020-05-23 11:35:01.984734
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 3818014, Time: 2020-05-23 11:36:55

Total Rows Read: 49900000, Paired Rows: 4642506, Time: 2020-05-23 12:20:22.263583
Total Rows Read: 50000000, Paired Rows: 4651437, Time: 2020-05-23 12:20:41.696655
Cleanin up!
Total Rows Read: 50100000, Paired Rows: 4658458, Time: 2020-05-23 12:22:55.972184
Total Rows Read: 50200000, Paired Rows: 4666634, Time: 2020-05-23 12:23:16.257743
Total Rows Read: 50300000, Paired Rows: 4675658, Time: 2020-05-23 12:23:36.277605
Total Rows Read: 50400000, Paired Rows: 4685139, Time: 2020-05-23 12:23:56.355975
Total Rows Read: 50500000, Paired Rows: 4694850, Time: 2020-05-23 12:24:15.858348
Total Rows Read: 50600000, Paired Rows: 4704539, Time: 2020-05-23 12:24:35.784028
Total Rows Read: 50700000, Paired Rows: 4713994, Time: 2020-05-23 12:24:55.060094
Total Rows Read: 50800000, Paired Rows: 4723581, Time: 2020-05-23 12:25:14.693803
Total Rows Read: 50900000, Paired Rows: 4733385, Time: 2020-05-23 12:25:34.000858
Total Rows Read: 51000000, Paired Rows: 4743209, Time: 2020-05-23 12:25:53.434078
Clea

In [10]:
df = open("../2015-05-data/RC_2015-05.bz2")

In [11]:
df

<_io.TextIOWrapper name='../2015-05-data/RC_2015-05.bz2' mode='r' encoding='UTF-8'>